In [1]:
import numpy as np
import pandas as pd
from dcddm import Ddm

np.random.seed(124)

In [2]:
###
#Generate data
###

(df, 
 mu_x_fix_cols, mu_x_rnd_cols, 
 w_x_fix_cols, w_x_rnd_cols,
 a_x_fix_cols, a_x_rnd_cols,
 nt_x_fix_cols, nt_x_rnd_cols) = Ddm().generate_data(N=100,T=5)

print(df)

 
Generating synthetic data for DDM.
     ind_id  obs_id  const      y        x1        x2        x3
0         1       1      1   1.91  0.106065  0.590954  0.293907
1         1       2      1   6.81  0.745471  0.842859  0.573189
2         1       3      1   9.05  0.572314  0.092416  0.760256
3         1       4      1   9.10  0.458241  0.521227  0.682589
4         1       5      1   3.05  0.384706  0.544023  0.270994
..      ...     ...    ...    ...       ...       ...       ...
495     100     496      1    NaN  0.331887  0.760527  0.099933
496     100     497      1    NaN  0.138803  0.092392  0.778268
497     100     498      1   2.26  0.753397  0.668311  0.049557
498     100     499      1  11.23  0.435243  0.699832  0.582726
499     100     500      1   5.46  0.478358  0.466973  0.838497

[500 rows x 7 columns]


In [3]:
###
#Estimate model
###

results = Ddm().estimate(
    df=df, 
    mu_x_fix_cols=mu_x_fix_cols, 
    mu_x_rnd_cols=mu_x_rnd_cols, 
    w_x_fix_cols=w_x_fix_cols, 
    w_x_rnd_cols=w_x_rnd_cols, 
    a_x_fix_cols=a_x_fix_cols, 
    a_x_rnd_cols=None, 
    n_draws=100, 
    compute_hess=False
    )

 
Estimation started at 2023-12-20 10:35:19


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.88702D+03    |proj g|=  5.19522D+03

At iterate    1    f=  1.72585D+03    |proj g|=  1.61162D+03

At iterate    2    f=  1.25440D+03    |proj g|=  3.40949D+02

At iterate    3    f=  1.21166D+03    |proj g|=  1.42025D+02

At iterate    4    f=  1.19819D+03    |proj g|=  4.43978D+01

At iterate    5    f=  1.19495D+03    |proj g|=  3.18494D+01

At iterate    6    f=  1.19382D+03    |proj g|=  2.53814D+01

At iterate    7    f=  1.19160D+03    |proj g|=  2.14784D+01

At iterate    8    f=  1.18737D+03    |proj g|=  3.89435D+01

At iterate    9    f=  1.18529D+03    |proj g|=  4.47793D+01

At iterate   10    f=  1.18301D+03    |proj g|=  2.60276D+01

At iterate   11    f=  1.18183D+03    |proj g|=  1.97249D+01

At iterate   12    f=  1.18153D+03    |proj g|=  1.41661D+01

At iterate   13    f=  1.1

In [4]:
###
#Elasticities
###

t_steps = np.arange(1,15+1)
probs_old = Ddm().predict(
    results,
    df, 
    ind_id_col='ind_id', 
    obs_id_col='obs_id', 
    y_col='y',
    mu_x_fix_cols=mu_x_fix_cols, 
    mu_x_rnd_cols=mu_x_rnd_cols, 
    w_x_fix_cols=w_x_fix_cols, 
    w_x_rnd_cols=w_x_rnd_cols, 
    a_x_fix_cols=a_x_fix_cols, 
    a_x_rnd_cols=None, 
    n_draws=100, 
    predict_loglik=False, t_steps=t_steps
    )

#print(pd.DataFrame(probs_old.reshape(-1,t_steps.shape[0]), columns=t_steps))

def arc_elasticity(df, x_name, delta, probs_old):
    df_new = df.copy()
    df_new[x_name] *= (1 + delta)

    probs_new = Ddm().predict(
        results,
        df_new, 
        ind_id_col='ind_id', 
        obs_id_col='obs_id', 
        y_col='y',
        mu_x_fix_cols=mu_x_fix_cols, 
        mu_x_rnd_cols=mu_x_rnd_cols, 
        w_x_fix_cols=w_x_fix_cols, 
        w_x_rnd_cols=w_x_rnd_cols, 
        a_x_fix_cols=a_x_fix_cols, 
        a_x_rnd_cols=None, 
        n_draws=100, 
        predict_loglik=False, t_steps=t_steps
        ) # n_ind, n_sit, n_steps
    
    elas_numer = (probs_new - probs_old) / ((probs_new + probs_old) / 2)
    x_old = df[x_name].values.reshape(probs_old.shape[0], probs_old.shape[1], 1)
    x_new = df_new[x_name].values.reshape(probs_old.shape[0], probs_old.shape[1], 1)
    elas_denom = (x_new - x_old) / ((x_new + x_old) / 2)
    elas = elas_numer / elas_denom

    return elas, probs_new

delta = 0.1
elas, probs_new = arc_elasticity(df, 'x1', delta, probs_old)
elas_mean = elas.mean(axis=(0,1))
#print(pd.DataFrame(probs_new.reshape(-1,t_steps.shape[0]), columns=t_steps))
#print(pd.DataFrame(elas.reshape(-1,t_steps.shape[0]), columns=t_steps))
print(pd.Series(elas_mean, index=t_steps))

    


1     0.334559
2     0.243655
3     0.215674
4     0.188694
5     0.162799
6     0.139617
7     0.119657
8     0.102879
9     0.089005
10    0.077674
11    0.068505
12    0.061139
13    0.055256
14    0.050575
15    0.046864
dtype: float64


In [5]:
###
# Response times and response time elasticities
###

rt_old = Ddm().predict(
    results,
    df, 
    ind_id_col='ind_id', 
    obs_id_col='obs_id', 
    y_col='y',
    mu_x_fix_cols=mu_x_fix_cols, 
    mu_x_rnd_cols=mu_x_rnd_cols, 
    w_x_fix_cols=w_x_fix_cols, 
    w_x_rnd_cols=w_x_rnd_cols, 
    a_x_fix_cols=a_x_fix_cols, 
    a_x_rnd_cols=None, 
    n_draws=100, 
    predict_loglik=False, predict_rt=True
    )

print(rt_old)

[[5.18252792 5.43347853 7.08268436 6.54163451 4.84333122]
 [7.21458315 6.0688187  7.75707456 4.76422812 3.85756957]
 [4.81796034 6.28621594 5.17891356 6.42639809 4.75176745]
 [6.13867925 4.46014872 4.30814559 7.04928559 6.70776788]
 [5.78613517 5.76213635 4.22748004 5.13153072 5.20898584]
 [5.50522443 5.65374868 3.44971697 6.56916824 6.59025215]
 [7.06811242 6.87860149 4.72521596 7.36802587 5.31681676]
 [6.06384745 6.6523402  6.89882229 5.74155455 6.36832961]
 [6.02815423 6.23699028 7.08477043 6.20080556 4.7619201 ]
 [4.22801331 3.83431819 4.76880393 6.53884789 4.86236604]
 [5.97643948 6.44292424 6.97422138 4.25161023 7.54836949]
 [4.16804082 5.46919364 4.93355877 7.9719028  4.73143551]
 [7.41132882 7.33062907 4.67722124 7.74996166 5.32574179]
 [5.57511885 6.92539243 6.00573253 7.86373509 6.94577391]
 [5.0199807  5.03107224 7.71753616 6.23861986 4.93049982]
 [7.12930008 5.92378805 5.16449569 6.83264875 7.44253181]
 [7.31361814 6.21659245 4.65335621 6.70237304 5.80405995]
 [7.76447194 5

In [6]:
def rt_arc_elasticity(df, x_name, delta, rt_old):
    df_new = df.copy()
    df_new[x_name] *= (1 + delta)

    rt_new = Ddm().predict(
        results,
        df_new, 
        ind_id_col='ind_id', 
        obs_id_col='obs_id', 
        y_col='y',
        mu_x_fix_cols=mu_x_fix_cols, 
        mu_x_rnd_cols=mu_x_rnd_cols, 
        w_x_fix_cols=w_x_fix_cols, 
        w_x_rnd_cols=w_x_rnd_cols, 
        a_x_fix_cols=a_x_fix_cols, 
        a_x_rnd_cols=None, 
        n_draws=100, 
        predict_loglik=False, predict_rt=True
        ) # n_ind, n_sit, n_steps
    
    elas_numer = (rt_new - rt_old) / ((rt_new + rt_old) / 2)
    x_old = df[x_name].values.reshape(rt_old.shape[0], rt_old.shape[1])
    x_new = df_new[x_name].values.reshape(rt_old.shape[0], rt_old.shape[1])
    elas_denom = (x_new - x_old) / ((x_new + x_old) / 2)
    elas = elas_numer / elas_denom

    return elas, rt_new

delta = 0.1
elas, rt_new = rt_arc_elasticity(df, 'x1', delta, rt_old)
elas_mean = elas.mean(axis=(0,1))
print(elas_mean)

-0.10767900395968559
